In [1]:
import numpy as np
from src.utils import *
from tqdm import tqdm

unary_operators=[np.sin, np.cos, np.exp, np.abs, np.log, np.tan]
binary_operators=[np.add, np.subtract, np.multiply, np.divide]


operators = unary_operators + binary_operators

In [2]:
problem_0 = np.load("data/problem_4.npz")
x = problem_0["x"]
y = problem_0["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)

x.shape: (2, 5000)
y.shape: (5000,)


In [3]:
NUM_POPULATION = 100
NUM_GENERATIONS = 1000
MAX_DEPTH_INITIAL = 5
DEDUPE_INTERVAL = 5
ELITISM = True
ELITE_COUNT = 3
TOURNAMENT_SUBSET_SIZE = 3
TOURNAMENT_WINNER_SIZE = 10
XOVER = 0.8
MUTATION = 0.8
BREED_NEW = 15
KILL_AGE = 15
CONSTANT_FIT_INTERVAL = 10
CONSTANT_FIT_ITERATION = 7

my_population = create_population(NUM_POPULATION,MAX_DEPTH_INITIAL,x.shape[0])
assign_population_fitness(my_population,x,y)
print(f" Population size: {len(my_population)}")
print(f"Population mean fitness: {calculate_mean_fitness(my_population)}")
print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")

# Sort the population by fitness (ascending), then take the top 5
top_5_individuals = top_n_individuals(my_population, 5)

# Print the top 5 individuals with the minimum fitness
for i, individual in enumerate(top_5_individuals, 1):
    print(f"Top {i}: Fitness = {individual.fitness}: Genome = {individual.genome}")

 Population size: 64
Population mean fitness: 3.3273168919634495e+28
Population mean complexity: 19.75
Top 1: Fitness = 20.664600300131436: Genome = cos(x[1])
Top 2: Fitness = 21.835749327762827: Genome = abs([1.53745754])
Top 3: Fitness = 21.882847158921958: Genome = ([0.71339317] - [-1.79712336])
Top 4: Fitness = 22.09932433529548: Genome = exp(cos(sin(abs(([-1.32218171] / [0.20573286])))))
Top 5: Fitness = 22.12834817947589: Genome = (cos(cos(sin((x[0] - [2.27156539])))) - cos(exp(((x[0] / x[0]) / abs([-1.27216841])))))


In [4]:

assign_population_fitness(my_population,x,y)
best_result = top_n_individuals(my_population, 1)[0]
for generation in tqdm(range(NUM_GENERATIONS)):
    num_xover=0
    num_mut=0
    num_succesfull_mut=0
    age_population(my_population)
    

    # Kill eldest individuals
    kill_eldest(my_population, KILL_AGE)
    # print("Population size before killing constants: ", len(my_population))
    
    # print("Population size after killing constants: ", len(my_population))


    best_ind = tournament_selection(my_population,TOURNAMENT_SUBSET_SIZE,TOURNAMENT_WINNER_SIZE,elitism=ELITISM, elite_count=ELITE_COUNT)

    
    for _ in range(BREED_NEW): # CHECK
        if random.random() < XOVER:
            # Xover the best individual and replace worst individuals
            parent1 = my_population[best_ind[np.random.choice(len(best_ind))]]
            parent2 = my_population[best_ind[np.random.choice(len(best_ind))]]
            child1, child2 = crossover(parent1,parent2)
            my_population.append(child1)
            my_population.append(child2)
            num_xover+=1

        if random.random() < MUTATION:
            # Mutate best individuals and replace worst individuals
            parent = my_population[best_ind[np.random.choice(len(best_ind))]]
            
            child, success = mutation_w_sa(parent, x.shape[0], x, y)
            num_mut+=1
            if success:                
                my_population.append(child)
                num_succesfull_mut+=1

    assign_population_fitness(my_population,x,y)
    # print(f"xover: {num_xover}, mut: {num_mut}, succesfull mut: {num_succesfull_mut}")

    # Update elites age as 0
    elites = top_n_individuals(my_population, ELITE_COUNT)
    if best_result.fitness > elites[0].fitness:
        best_result = elites[0]
        print(f"New best result found: {best_result}")
    
    for elite in elites:
        my_population.remove(elite)
        elite.age = 0
        my_population.append(elite)

        
    
    # best_10 = top_n_individuals(my_population, 20)
    # for ind in best_10:
    #     print(f" Genome: {ind.genome}")

    # print("Population mean complexity: ", calculate_mean_complexity(my_population))
    simplify_population(my_population)
    # print("Population mean complezity after simplifying: ", calculate_mean_complexity(my_population))
    # best_10 = top_n_individuals(my_population, 20)
    # for ind in best_10:
    #     print(f" Genome: {ind.genome}")

    # Deduplicate every few iterations
    if generation % DEDUPE_INTERVAL == DEDUPE_INTERVAL-1:
        print("Population size before Deduplicating population", len(my_population))
        my_population = deduplicate_population(my_population)
        print("Population size after deduplicating: ", len(my_population))
        kill_constant(my_population)
        print("Population size after killing constants: ", len(my_population))
        print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")
        best_10 = top_n_individuals(my_population, 5)
        for ind in range(len(best_10)):
            print(f"{ind}. Fitness: {best_10[ind].fitness:.3f}, Genome: {best_10[ind].genome}, Age: {best_10[ind].age}")
        print()
    #     # Fit constants of every individual
    # if generation % CONSTANT_FIT_INTERVAL == CONSTANT_FIT_INTERVAL-1:
    #     print(f"Generation {generation}: Mean fitness {calculate_mean_fitness(my_population)}")
    #     for individual in my_population:
    #         stronger_inv = fit_constants(individual,CONSTANT_FIT_ITERATION, x, y)
    #         my_population.remove(individual)
    #         my_population.append(stronger_inv)
    #     print(f"Population mean fitness after constant fit: {calculate_mean_fitness(my_population)}")
        
my_population = deduplicate_population(my_population)

  0%|          | 4/1000 [01:10<4:54:24, 17.74s/it]

Population size before Deduplicating population 238


TypeError: deduplicate_population() got an unexpected keyword argument 'p'

In [ ]:
assign_population_fitness(my_population,x,y)
# Sort the population by fitness (ascending), then take the top 5
top_5_individuals = top_n_individuals(my_population, 15)

# Print the best individual
print(f"Best individual: Fitness = {best_result.fitness}: Genome = {best_result.genome}")
# Print the top 5 individuals with the minimum fitness in population
for i, individual in enumerate(top_5_individuals, 1):
    print(f"Top {i}: Fitness = {individual.fitness}: Genome = {individual.genome}")

# Print the mean fitness of the population
print("Mean fitness of the population: ", calculate_mean_fitness(my_population))
print("Population size: ", len(my_population))
print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")

In [ ]:
top = top_n_individuals(my_population, len(my_population))
top[-30:]

In [ ]:
top[:30]

In [ ]:
problem_4 = np.load("data/problem_4.npz")
x = problem_4["x"]
y = problem_4["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)

In [ ]:
NUM_POPULATION = 1000
NUM_GENERATIONS = 10_000
DEDUPE_INTERVAL = 10
MAX_DEPTH_INITIAL = 6
ELITISM = True
ELITE_COUNT = 7
TOURNAMENT_SUBSET_SIZE = 3
TOURNAMENT_WINNER_SIZE = 50
XOVER = 0.8
MUTATION = 0.8
BREED_NEW = 100
KILL_AGE = 20

population_4 = create_population(NUM_POPULATION,MAX_DEPTH_INITIAL,x.shape[0])
assign_population_fitness(population_4,x,y)
print(f" Population size: {len(population_4)}")
print(f"Population mean fitness: {calculate_mean_fitness(population_4)}")
print(f"Population mean complexity: {calculate_mean_complexity(population_4)}")

# Sort the population by fitness (ascending), then take the top 5
top_5_individuals = top_n_individuals(population_4, 5)

# Print the top 5 individuals with the minimum fitness
for i, individual in enumerate(top_5_individuals, 1):
    print(f"Top {i}: Fitness = {individual.fitness}: Genome = {individual.genome}")

In [ ]:

operators=[np.add, np.subtract, np.multiply, np.sin, np.cos, np.exp]
unary_operators=[np.sin, np.cos, np.exp]

assign_population_fitness(population_4,x,y)
best_result = top_n_individuals(population_4, 1)[0]
for generation in tqdm(range(NUM_GENERATIONS)):
    num_xover=0
    num_mut=0
    num_succesfull_mut=0
    age_population(population_4)
    

    # Kill eldest individuals
    kill_eldest(population_4, KILL_AGE)
    # print("Population size before killing constants: ", len(population_4))
    kill_constant(population_4)
    # print("Population size after killing constants: ", len(population_4))


    best_ind = tournament_selection(population_4,TOURNAMENT_SUBSET_SIZE,TOURNAMENT_WINNER_SIZE,elitism=ELITISM, elite_count=ELITE_COUNT)

    
    for _ in range(BREED_NEW): # CHECK
        if random.random() < XOVER:
            # Xover the best individual and replace worst individuals
            parent1 = population_4[best_ind[np.random.choice(len(best_ind))]]
            parent2 = population_4[best_ind[np.random.choice(len(best_ind))]]
            child1, child2 = crossover(parent1,parent2)
            population_4.append(child1)
            population_4.append(child2)
            num_xover+=1

        if random.random() < MUTATION:
            # Mutate best individuals and replace worst individuals
            parent = population_4[best_ind[np.random.choice(len(best_ind))]]
            
            child, success = mutation_w_sa(parent, x.shape[0], x, y)
            num_mut+=1
            if success:                
                population_4.append(child)
                num_succesfull_mut+=1

    assign_population_fitness(population_4,x,y)
    # print(f"xover: {num_xover}, mut: {num_mut}, succesfull mut: {num_succesfull_mut}")

    current_best_result = top_n_individuals(population_4, 1)[0]
    if best_result.fitness > current_best_result.fitness:
        best_result = current_best_result
        print(f"New best result found: {best_result}")
    

        
    # Deduplicate every few iterations
    if generation % DEDUPE_INTERVAL == DEDUPE_INTERVAL-1:
        # print("Population size before Deduplicating population", len(population_4))
        population_4 = deduplicate_population(population_4, p=0)
        # print("Population size after deduplicating: ", len(population_4))
        # print(f"Population mean complexity: {calculate_mean_complexity(population_4)}")
population_4 = deduplicate_population(population_4)

In [ ]:
assign_population_fitness(population_4,x,y)
# Sort the population by fitness (ascending), then take the top 5
top_5_individuals = top_n_individuals(population_4, 15)

# Print the best individual
print(f"Best individual: Fitness = {best_result.fitness}: Genome = {best_result.genome}")
# Print the top 5 individuals with the minimum fitness in population
for i, individual in enumerate(top_5_individuals, 1):
    print(f"Top {i}: Fitness = {individual.fitness}: Genome = {individual.genome}")

# Print the mean fitness of the population
print("Mean fitness of the population: ", calculate_mean_fitness(population_4))
print("Population size: ", len(population_4))
print(f"Population mean complexity: {calculate_mean_complexity(population_4)}")

In [10]:
best_sol = Node(value=np.add, left=Node(feature_index=0), right=Node(np.multiply, left=Node(feature_index=1), right=Node(value=0.2)))

In [ ]:
print("Best solution: ", best_sol)

In [ ]:
cost(best_sol, x, y)